# Tutorial 4: Making Maps with Cartopy

So far, we've learned how to plot data using x and y axes coordinates; basically, we can make line plots and scatter plots. Another useful tool for showing atmospheric and environmental data is plotting the data in space, i.e. making a map. 

In this tutorial, we will learn about the map-making package Cartopy. By the end of this tutorial, you will be able to:
* create a map background of an area of interest
* customize a map for your data
* display data on a map for multiple map projections

### What is Cartopy?

Cartopy is package the focuses on geospatial data. This means it can be used for plotting data on a map, but it can also be used for plotting geographical images such as sections of a Google map. All the info about Cartopy can be found here: https://scitools.org.uk/cartopy/docs/latest/

Like Matplotlib, Cartopy has several different modules. We will be working with two important modules in this tutorial: *crs* and *feature*. crs is the basic map-making module that creates the coordinates that we want. feature allows us to add coastlines, borders, oceans, and other important distinctions to our maps. We'll work through these modules in detail in this tutorial. Then, once we've made the map background, we actually use Matplotlib to plot the data. 

For this tutorial, we will again be using pandas to manipulate our data. We will use Matplotlib for the plotting of data on the Cartopy-created map backgrounds. 

### Data Preparation

**Note:** If you get a <font color=red>ModuleNotFoundError</font> when importing Cartopy modules, then you will need to install this package. First, close and halt this notebook and quit jupyter. Then, in either anaconda prompt (PC) or terminal (mac), type "conda install cartopy". Once that is installed, restart this notebook and try again!

In [ ]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from cartopy import crs, feature # we only want the crs and feature modules from cartopy

In [ ]:
# we'll be using some new data this time
data = pd.read_csv('july20_ozone.csv')

Take a moment to examine this data on your own. Use the techniques you learned in tutorials 2 and 3. Some good first steps:
1. print the column names
2. print the shape of the data
3. print the first few rows
4. look at the data

In [ ]:
# what are the column names? type code in this cell

In [ ]:
# what is the shape/size of the data? type code in this cell

In [ ]:
# what do the first few rows of data look like? what type of data is there? type code in this cell

**Knowledge Check** Based on your explorations, what is the y-data? What are different ways we could organize or subselect the data?

Let's now go through step 4 together to examine the data more.

In [ ]:
# first, to simplify some stuff later, we'll first make sure the date column is in datetime format
dates = pd.to_datetime(data.loc[:,'Date Local'])

In [ ]:
# add the datetimes back to the DataFrame
data['datetimes'] = dates

In [ ]:
# check how the data type of "Date Local" column changed 
print(type(data['Date Local'][0]))
print(type(data['datetimes'][0]))

In [ ]:
# one way we can examine the data is by looking at trends in time
# get the average of each day of the month using groupby
all_dayave = data.groupby(data['datetimes'].dt.day).mean()

In [ ]:
# look at the data. What changed about the columns? Does anything seem weird with the lat and lon values?
all_dayave

In [ ]:
# let's just do a quick visualization of the data
ydata = all_dayave['Ozone (ppm)']
xdata = all_dayave.index

fig = plt.figure(figsize=(8,6))
plt.plot(xdata,ydata)
plt.xlabel('Day of the Month')
plt.ylabel('Ozone (ppm)')
plt.title('July Daily Average Ozone')
plt.show()

This plot is telling us that the average ozone concentration across the entire United States fluctuates by less than 10 ppb during the month of July. Does this make sense? Since we are averaging across a very large area, it does! The high and low values will cancel out. But what is happening at specific locations? Can we zoom in to an area to better understand how air quality is changing there?

In [ ]:
# here, we subselect the data based on state
# you can change the state name to another state if you're interested, 
# just be careful with some of the code later on!
il = data.loc[data['State Name'] == 'Illinois']
il

In [ ]:
# one way we can examine the data is by looking at trends in time
# get the average of each day of the month using groupby
il_dayave = il.groupby(il['datetimes'].dt.day).mean()

In [ ]:
# let's just do a quick visualization of the data
ydata = il_dayave['Ozone (ppm)']
xdata = il_dayave.index

fig = plt.figure(figsize=(8,6))
plt.plot(xdata,ydata)
plt.xlabel('Day of the Month')
plt.ylabel('Ozone (ppm)')
plt.title('July Daily Average Ozone in Illinois')
plt.show()

Ok, we can see that the ozone concentration is decreasing during July in Illinois. But how does this break down by monitoring location? Are there differences between monitors? To investigate this, we can make a map to see which monitors have high concentrations and which have low concentrations.

### Making a Map

There are a few steps to making a map. 
1. Identify the coordinates of your area of interest
2. Make a grid on your coordinates of interest
3. Select a map projection
4. Create a pyplot figure using your grid and map projection
5. Add the relevant features to your figure
6. Plot the data

#### 1. Identify the coordinates
In some cases, your data will have a pre-made grid of the latitude and longitude coordinates for you. This makes it really easy. In other cases, you might need to decide on the coordinates yourself. Our data does have latitude and longitude coordinates, but it does not tell us the size of the grid we want. We can use our latitude and longitude data to make an appropriately sized grid.

In [ ]:
latmin = il['Latitude'].min()
latmax = il['Latitude'].max()
lonmin = il['Longitude'].min()
lonmax = il['Longitude'].max()
print('Lat Range: %.4f - %.4f' % (latmin,latmax))
print('Lon Range: %.4f - %.4f' % (lonmin,lonmax))

Now we know the max and min values of our coordinates. This will help us to make a box around our data. It is best to round up/down to give a little bit of buffer space around the data.

In [ ]:
# buffered lat/lon values
latmin = 36
latmax = 44
lonmin = -85
lonmax = -93

#### 2. Make a grid

This step will vary depending on the type of data you are plotting. We will be making a simple dot map, so we basically need to set a max and min range for our map's box size. If we were using gridded data that covered the whole area, like satellite data or a model output, we would need to make a meshgrid that had a lat and lon coordinate pair for each grid cell. We will go through that process in a later tutorial.

In [ ]:
# set the extents of our box
extent = [lonmin,lonmax,latmin,latmax]

#### 3. Select a map projection

We also need to decide on a map projection. A map projection tells you how the map will be displayed. Check out this website for examples: https://scitools.org.uk/cartopy/docs/latest/crs/projections.html

In [ ]:
# let's set our projection as PlateCarree
proj = crs.PlateCarree()

#### 4. Create a figure

Now that we have the basics ready, we can put them together in a pyplot figure to set up our map background. 

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
# ax.set_extent([lonmin,lonmax,latmin,latmax],crs.PlateCarree())
ax.set_extent(extent)
# # add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
plt.show()

Woot! A box! How can we tell if this box is set up correctly? We can add feature details like rivers, state borders, etc. to determine if it's in the right place.

#### 5. Add relevant features

**Note:** Depending on which version of Cartopy you have installed, you may get some download warnings or errors with the features. 
* If you get errors and the map does not show, exit this notebook and run a "conda install --update-all cartopy" command in either terminal (Mac) or anaconda prompt (Windows).
* If you are getting DownloadWarnings but the figure does appear, then don't worry. If you rerun the cell, you will see the DownloadWarning has disappeared. If you add new features, you will get a new warning, but it'll go away the next time you use that feature. When Cartopy was initially installed, it didn't download all the different map features with it. So it has to download them the first time you want to use those features in a plot.

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.RIVERS)
plt.show()

Success! That is definitely a box around Illinois! If you want to change the area of the map, go back to step 1 and select new lat and lon values. 

Ok, now to plot the data!

#### 6. Plot the data

The daily ozone data has lat and lon coordinates. But we have data for every day in the month of July, which is too much to plot all at once. Instead, let's pick just one day and plot that data first. 

In [ ]:
# select monitor data from July 1st
first_day = il[il['datetimes'] == '2020-07-01']

In [ ]:
# check what it looks like
first_day.head()

In [ ]:
# get the lat, lon, and y-data
lats = first_day['Latitude']
lons = first_day['Longitude']
ozone = first_day['Ozone (ppm)']

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
# ax.add_feature(feature.RIVERS)
# plot the data -- note, we use plt.scatter to add data on top of the map!
plt.scatter(lons,lats,s=50,c=ozone)
plt.colorbar(label='Ozone (ppm)',shrink=0.5)
plt.show()

Awesome! We've map a map, plotted the data, and can now start doing some analysis!

This map makes it very clear that the high concentrations of ozone happen close to Lake Michigan, and the low concentrations of ozone in the state are further south. But this is only for one day. Let's repeat the process with data from another day.

In [ ]:
# select monitor data from July 31st
last_day = il.loc[il['datetimes'] == '2020-07-31']

# get the lat, lon, and y-data
lats = last_day['Latitude']
lons = last_day['Longitude']
ozone = last_day['Ozone (ppm)']

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
# ax.add_feature(feature.RIVERS)
# plot the data -- note, we use plt.scatter to add data on top of the map!
plt.scatter(lons,lats,s=50,c=ozone)
plt.colorbar(label='Ozone (ppm)',shrink=0.5)
plt.show()

Well, that's different! There are two things you might have noticed right away. First, the high and low values have changed monitor locations. Second, the colorbar scale is much lower. On July 31st, the ozone is no longer at a peak around Lake Michigan and, overall, ozone concentrations have decreased across the state. This is really interesting, and we wouldn't have learned this from only looking at the line plot. This highlights the importance of using maps to visualize spatial data. But it also reminds us that things change over time, so we can't just look at one instant of data! 

### The importance of projections

Now that we've learned how to make a map, let's dig a little deeper into some of the details of Cartopy. Specifically, what is going on with the map projections? What do different projections look like? How does this affect the data?

To evaluate the importance of projections, we really need to look on a larger scale than the state level. Since our ozone data is for the entire US, let's look at the total area covered by the data.

In [ ]:
# what are the min and max coordinates of monitors in our dataset?
latmin = data['Latitude'].min()
latmax = data['Latitude'].max()
lonmin = data['Longitude'].min()
lonmax = data['Longitude'].max()
print('Lat Range: %.4f - %.4f' % (latmin,latmax))
print('Lon Range: %.4f - %.4f' % (lonmin,lonmax))

That's a pretty big range! Let's go through steps 1-5 of making a map, but looking at some different projections.

In [ ]:
# buffered lat and lon values
latmin = 15
latmax = 65
lonmin = -60
lonmax = -150

In [ ]:
# set the extents of our box
extent = [lonmin,lonmax,latmin,latmax]

In [ ]:
# first, we'll stick with PlateCarree
proj = crs.PlateCarree()

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
# ax.add_feature(feature.RIVERS)
plt.show()

Ha! You might have noticed something funny about this figure. We see state borders, but not countries! Let's fix that.

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
# ax.add_feature(feature.RIVERS)
ax.add_feature(feature.COASTLINE)
plt.show()

That's definitely better. What do you notice about the shape of this map? Does the US seem to be the right shape? Or do things look a bit flattened? That's what happens with PlateCarree, it flattens out the map. Let's try some other projections. 

**Note:** If you get '<font color=red>TypeError</font>: Cannot label gridlines on a LambertConformal plot. Only PlateCarree and Mercator plots are currently supported.' then you need to update your version of Cartopy. Exit this notebook and run a "conda install --update-all cartopy" command in either terminal (Mac) or anaconda prompt (Windows).

In [ ]:
# Albers Equal Area projection
proj = crs.AlbersEqualArea()

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
# ax.add_feature(feature.RIVERS)
ax.add_feature(feature.COASTLINE)
plt.show()

That's pretty funky! The Albers Equal Area projection is a conical projection, meaning it pretends the globe is actually a cone. This is how it accounts for curvature on our flat computer screens.

In [ ]:
# Lambert Conformal. This projection is useful if you ever use CMAQ or WRF data
proj = crs.LambertConformal()

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
# ax.add_feature(feature.RIVERS)
ax.add_feature(feature.COASTLINE)
plt.show()

Again, there is curvature in the map. But this time the US is centered a bit better. 

In [ ]:
# Robinson projection
proj = crs.Robinson()

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.RIVERS)
ax.add_feature(feature.COASTLINE)
plt.show()

Another interesting interpretation of curvature. This projection is better for mapping the whole earth, not just sections of it.

There are many other map projections. Which one you use really depends on your data and what you want to show. Here is the full list with images: https://scitools.org.uk/cartopy/docs/latest/crs/projections.html

Ok, so we've examined a few projections. Let's see how this can affect the data. Let's pick just one random day to focus on, and plot all monitor locations.

In [ ]:
# let's map July 4th, 2020
sel_date = data[data['datetimes'] == '2020-07-04']

In [ ]:
# get the lat, lon, and y-data
lats = sel_date['Latitude']
lons = sel_date['Longitude']
ozone = sel_date['Ozone (ppm)']

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.RIVERS)
ax.add_feature(feature.COASTLINE)
# add data
plt.scatter(lons,lats,s=10,c=ozone)
plt.colorbar(label='Ozone (ppm)',shrink=0.5)
plt.show()

Wait! What happened? Where is the data? The lats, lons, and ozone variables definitely have values, so that isn't the problem. Why did the dots show up on the map?

The answer is in the projection. We changed the map projection from Plate Carree to Robinson. The data is in Plate Carree (flat) projection. In order to map our flat data onto a not-flat map, we need to transform it. 

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.RIVERS)
ax.add_feature(feature.COASTLINE)
# add data
plt.scatter(lons,lats,s=10,c=ozone,transform=crs.PlateCarree()) # added transform here
plt.colorbar(label='Ozone (ppm)',shrink=0.5)
plt.show()

Ta da! We have now transformed our flat data so that it can be plotted on a curved map. Basically any time you plot data onto a large map using a non-flat projection, you will need to add a transformation. 

### Exercises

**1.** Now it's your turn! Pick any state in te US and go through steps 1-6 to plot the data on a map. You can make the map as large or small as you want, but make sure you capture ALL the monitors in that state. Some states will be easier than others. 

**2.** Let's zoom out again. Make a map of the United States using a new projection from https://scitools.org.uk/cartopy/docs/latest/crs/projections.html. Then plot the July average ozone concentrations for all monitors across the US. Do you notice anything interesting?